In [ ]:
import spotipy
import pandas as pd
# Show all columns in dataframe
pd.set_option('display.max_columns', None)
from spotipy.oauth2 import SpotifyClientCredentials
from datetime import datetime
import dateutil

In [ ]:
#Authentication
client_credentials_manager = SpotifyClientCredentials(client_id='0c78fdc53b5d482992b86a7f00fc434f', client_secret='924ef3f4065d47ffbd22dfe4939c8625')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
def get_playlist_URI(playlist_URL):
    """
    Get the spotify uri from the url:
    beware as there's no error handling
    """
    return playlist_URL.split("/")[-1].split("?")[0]

In [ ]:
def get_playlist_tracks(playlist_URI):
    results = sp.playlist_tracks(playlist_URI)
    tracks_data = []

    # Define lookup tables to convert key and mode ints to strings
    track_key = {
        -1: '',
        0: 'C',
        1: 'C#',
        2: 'D',
        3: 'D#',
        4: 'E',
        5: 'F',
        6: 'F#',
        7: 'G',
        8: 'G#',
        9: 'A',
        10: 'A#',
        11: 'B'
    }
    track_mode = {
        0: 'Minor',
        1: 'Major'
    }
    
    for track in results['items']:
        # might want to consider moving audio_features call outside the loop
        # to avoid rate limiting (429 response)
        audio_features = sp.audio_features(track['track']['id'])[0]

        # Extract release_date so it can be separated into year/month/day
        album_release_date = dateutil.parser.parse(track['track']['album']['release_date'], default = datetime(1900, 1, 1))
        
        track_data = {
            'Track Name': track['track']['name'],
            'Artist(s)': ', '.join([artist['name'] for artist in track['track']['artists']]),
            'Artist Count': len(track['track']['artists']),
            'Release Year': album_release_date.year,
            'Release Month': album_release_date.month,
            'Release Day': album_release_date.day,
            'Track Popularity': track['track']['popularity'],
            'Acousticness': audio_features['acousticness'],
            'Danceability': audio_features['danceability'],
            'Duration (ms)': track['track']['duration_ms'],
            'Energy': audio_features['energy'],
            'Explicit': track['track']['explicit'],
            'Instrumentalness': audio_features['instrumentalness'],
            'Key': track_key[audio_features['key']],
            'Liveness': audio_features['liveness'],
            'Loudness (dB)': audio_features['loudness'],
            'Mode': track_mode[audio_features['mode']],
            'Speechiness': audio_features['speechiness'],
            'Tempo (bpm)': audio_features['tempo'],
            'Time Signature (/4)': audio_features['time_signature'],
            'Valence': audio_features['valence'],
        }
        tracks_data.append(track_data)
    return tracks_data

In [ ]:
top_songs_global_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
top_songs_global_URI = get_playlist_URI(top_songs_global_link)

data = pd.DataFrame(get_playlist_tracks(top_songs_global_URI))
data.head()

,Track Name,Artist(s),Artist Count,Release Year,Release Month,Release Day,Track Popularity,Acousticness,Danceability,Duration (ms),Energy,Explicit,Instrumentalness,Key,Liveness,Loudness (dB),Mode,Speechiness,Tempo (bpm),Time Signature (/4),Valence
0,La Diabla,Xavi,1,2023,11,30,99,0.2550,0.751,172264,0.819,False,0.000000,B,0.2360,-4.372,Minor,0.0409,131.842,3,0.697
1,greedy,Tate McRae,1,2023,9,15,100,0.2560,0.750,131872,0.733,True,0.000000,F#,0.1140,-3.180,Minor,0.0319,111.018,1,0.844
2,"One Of The Girls (with JENNIE, Lily Rose Depp)","The Weeknd, JENNIE, Lily-Rose Depp",3,2023,6,23,98,0.0545,0.522,244684,0.641,False,0.000000,G#,0.1620,-6.287,Major,0.0318,87.510,4,0.202
3,Lovin On Me,Jack Harlow,1,2023,11,10,99,0.0026,0.943,138411,0.558,True,0.000002,D,0.0937,-4.911,Major,0.0568,104.983,4,0.606
4,My Love Mine All Mine,Mitski,1,2023,9,15,99,0.8680,0.504,137773,0.308,False,0.135000,A,0.1580,-14.958,Major,0.0321,113.950,4,0.121


In [ ]:
top_songs_global_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
top_songs_global_URI = playlist_link.split("/")[-1].split("?")[0]
top_songs_global = sp.playlist_tracks(top_songs_global_URI, limit=100)
sp.audio_features(top_songs_global['items'][0]['track']['id'])